In [1]:
import pandas as pd
import pymysql
import boto3

from smart_open import open as s_open
from easydict import EasyDict
from dotenv import load_dotenv

from datetime import datetime
import time
import os

load_dotenv()

True

In [2]:
settings = EasyDict()

settings.AWS_ACCESS_KEY_ID = os.getenv('settings.AWS_ACCESS_KEY_ID')
settings.AWS_SECRET_ACCESS_KEY=os.getenv("settings.AWS_SECRET_ACCESS_KEY")
settings.AWS_REGION_NAME=os.getenv("settings.AWS_REGION_NAME")
settings.AWS_ACCOUNT_ID=os.getenv("settings.AWS_ACCOUNT_ID")

settings.DB_HOST=os.getenv("settings.DB_HOST")
settings.DB_USER=os.getenv("settings.DB_USER")
settings.DB_PASSWORD=os.getenv("settings.DB_PASSWORD")
settings.DB_NAME=os.getenv("settings.DB_NAME")
settings.DB_PORT=os.getenv("settings.DB_PORT")

settings.AWS_BUCKET_NAME="genia-bucket"
settings.AWS_ATHENA_OUTPUT_LOCATION="athena/quries"
settings.AWS_ATHENA_DATABASE="mini_db"

In [3]:
class Boto3Client(object):
    aws_access_key_id = settings.AWS_ACCESS_KEY_ID
    aws_secret_access_key = settings.AWS_SECRET_ACCESS_KEY
    region_name = settings.AWS_REGION_NAME
    bucket_name = settings.AWS_BUCKET_NAME
    athena_database=settings.AWS_ATHENA_DATABASE
    athena_output_location=settings.AWS_ATHENA_OUTPUT_LOCATION
    
    service_name = None
    
    @classmethod
    def get_client(cls):
        options = dict(
            aws_access_key_id=Boto3Client.aws_access_key_id,
            aws_secret_access_key=Boto3Client.aws_secret_access_key,
            region_name=Boto3Client.region_name,
        )
        return boto3.client(cls.service_name, **options)

In [4]:
class S3Client(Boto3Client):
    service_name = "s3"
    
    @staticmethod
    def get_s3_df(file_name: str):
        
        clnt = S3Client.get_client()
        obj = clnt.get_object(
                Bucket=S3Client.bucket_name,
                Key=file_name
            )
        return pd.read_csv(obj["Body"])
    
    @staticmethod
    def upload_s3_df(df: pd.DataFrame, file_name: str):
        try:
            clnt = S3Client.get_client()
            file_name = f"s3://{S3Client.bucket_name}/{file_name}"
            with s_open(file_name, "wb", transport_params=dict(client=clnt)) as out_file:
                df.to_parquet(out_file, engine="pyarrow", compression="gzip", index=False)
            return True
        except Exception as e:
            print("Error occured: ", str(e))
            return False

In [5]:
class AthenaClient(Boto3Client):
    
    service_name = "athena"
    
    output_location = f"s3://{Boto3Client.bucket_name}/{Boto3Client.athena_output_location}"
    
    @staticmethod
    def get_athena_query_exec_id(sql: str):
        
        clnt = AthenaClient.get_client()
        response = clnt.start_query_execution(
            QueryString=sql,
            QueryExecutionContext={"Database": AthenaClient.athena_database},
            ResultConfiguration={"OutputLocation": AthenaClient.output_location},
        )
        # response 내의 StatusCode == 200 확인
        return response["QueryExecutionId"]
    
    @staticmethod
    def collect_query_result(query_exec_id: str):
        
        clnt = AthenaClient.get_client()
        
        WAIT = ["QUEUED", "RUNNING"]
        SUCCESS = ["SUCCEEDED"]
        FAILED = ["FAILED", "CANCELLED"]
        
        while True:
            try:
                result = clnt.get_query_execution(QueryExecutionId=query_exec_id)
                status = result["QueryExecution"]["Status"]["State"]

                if status in SUCCESS:
                    query_result_path = f"{AthenaClient.output_location}/{query_exec_id}.csv"
                    query_result_path = query_result_path.replace(f"s3://{AthenaClient.bucket_name}/", "")
                    return S3Client.get_s3_df(query_result_path)

                if status in FAILED:
                    print(f"FAILED!!! -> {status}")
                    break

                if status in WAIT:
                    print(f"Still Running... -> {status}")
                    time.sleep(0.5)
                    continue

                print(f"unexpected status... -> {status}")
                break

            except Exception as e:
                print(str(e))
                break
        return False
    
    @staticmethod
    def get_athena_sql(sql: str):
        query_exec_id = AthenaClient.get_athena_query_exec_id(sample_sql)
        return AthenaClient.collect_query_result(query_exec_id)

In [6]:
sample_sql = 'SELECT * FROM "mini_db"."teacher-hotel"'
result = AthenaClient.get_athena_sql(sample_sql)

result

Still Running... -> QUEUED
Still Running... -> RUNNING
Still Running... -> RUNNING
Still Running... -> RUNNING
Still Running... -> RUNNING
Still Running... -> RUNNING


,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date,yyyy,mm
0,Resort Hotel,0,109,2016,January,1,1,0,1,2,...,NaN,0,Transient-Party,59.94,0,1,Check-Out,2016-01-02,2016,1
1,Resort Hotel,0,109,2016,January,1,1,0,1,2,...,NaN,0,Transient-Party,116.10,1,1,Check-Out,2016-01-02,2016,1
2,Resort Hotel,1,2,2016,January,1,1,0,1,2,...,NaN,0,Transient,89.00,0,1,No-Show,2016-01-01,2016,1
3,Resort Hotel,0,88,2016,January,1,1,0,2,2,...,NaN,0,Transient,73.46,0,2,Check-Out,2016-01-03,2016,1
4,Resort Hotel,1,20,2016,January,1,1,0,2,2,...,NaN,0,Transient,119.00,0,0,Canceled,2015-12-22,2016,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114460,City Hotel,0,2,2017,March,10,9,8,19,0,...,NaN,0,Transient,0.00,0,0,Check-Out,2017-04-05,2017,3
114461,City Hotel,0,58,2017,March,13,29,2,7,2,...,NaN,0,Transient,80.76,0,0,Check-Out,2017-04-07,2017,3
114462,City Hotel,0,152,2017,March,12,25,4,10,2,...,NaN,0,Transient,120.90,0,2,Check-Out,2017-04-08,2017,3
114463,City Hotel,0,53,2017,March,13,29,2,8,2,...,NaN,0,Transient,80.80,0,0,Check-Out,2017-04-08,2017,3


In [7]:
hotel = result.copy()
hotel.head()

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date,yyyy,mm
0,Resort Hotel,0,109,2016,January,1,1,0,1,2,...,NaN,0,Transient-Party,59.94,0,1,Check-Out,2016-01-02,2016,1
1,Resort Hotel,0,109,2016,January,1,1,0,1,2,...,NaN,0,Transient-Party,116.10,1,1,Check-Out,2016-01-02,2016,1
2,Resort Hotel,1,2,2016,January,1,1,0,1,2,...,NaN,0,Transient,89.00,0,1,No-Show,2016-01-01,2016,1
3,Resort Hotel,0,88,2016,January,1,1,0,2,2,...,NaN,0,Transient,73.46,0,2,Check-Out,2016-01-03,2016,1
4,Resort Hotel,1,20,2016,January,1,1,0,2,2,...,NaN,0,Transient,119.00,0,0,Canceled,2015-12-22,2016,1


# Hotel Preprocessing

In [8]:
hotel.shape

(114465, 34)

In [9]:
hotel.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114465 entries, 0 to 114464
Data columns (total 34 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   hotel                           114465 non-null  object 
 1   is_canceled                     114465 non-null  int64  
 2   lead_time                       114465 non-null  int64  
 3   arrival_date_year               114465 non-null  int64  
 4   arrival_date_month              114465 non-null  object 
 5   arrival_date_week_number        114465 non-null  int64  
 6   arrival_date_day_of_month       114465 non-null  int64  
 7   stays_in_weekend_nights         114465 non-null  int64  
 8   stays_in_week_nights            114465 non-null  int64  
 9   adults                          114465 non-null  int64  
 10  children                        114461 non-null  float64
 11  babies                          114465 non-null  int64  
 12  meal            

In [10]:
print(type(hotel['reservation_status_date'][0]), hotel['reservation_status_date'][0])

<class 'str'> 2016-01-02


In [11]:
hotel["reservation_status_date"] = pd.to_datetime(hotel["reservation_status_date"])
print(type(hotel['reservation_status_date'][0]), hotel['reservation_status_date'][0])

<class 'pandas._libs.tslibs.timestamps.Timestamp'> 2016-01-02 00:00:00


In [12]:
hotel.arrival_date_year.astype(str) + '/' + hotel.arrival_date_month.astype(str) + '/' + hotel.arrival_date_day_of_month.astype(str)

0         2016/January/1
1         2016/January/1
2         2016/January/1
3         2016/January/1
4         2016/January/1
               ...      
114460      2017/March/9
114461     2017/March/29
114462     2017/March/25
114463     2017/March/29
114464     2017/March/31
Length: 114465, dtype: object

In [13]:
hotel['arrival_date'] = pd.to_datetime(hotel.arrival_date_year.astype(str) + '/' + 
                                       hotel.arrival_date_month.astype(str) + '/' + 
                                       hotel.arrival_date_day_of_month.astype(str))

In [14]:
hotel['arrival_date']

0        2016-01-01
1        2016-01-01
2        2016-01-01
3        2016-01-01
4        2016-01-01
            ...    
114460   2017-03-09
114461   2017-03-29
114462   2017-03-25
114463   2017-03-29
114464   2017-03-31
Name: arrival_date, Length: 114465, dtype: datetime64[ns]

## 1. Finding the number of missing values in every column

In [15]:
import numpy as np
np.sum(hotel.isnull())

hotel                                  0
is_canceled                            0
lead_time                              0
arrival_date_year                      0
arrival_date_month                     0
arrival_date_week_number               0
arrival_date_day_of_month              0
stays_in_weekend_nights                0
stays_in_week_nights                   0
adults                                 0
children                               4
babies                                 0
meal                                   0
country                              488
market_segment                         0
distribution_channel                   0
is_repeated_guest                      0
previous_cancellations                 0
previous_bookings_not_canceled         0
reserved_room_type                     0
assigned_room_type                     0
booking_changes                        0
deposit_type                           0
agent                              16048
company         

## 2. To find the indexes of the missing value

In [16]:
# nan값 찾기
hotel.children[hotel.children != hotel.children]

93340   NaN
93407   NaN
93419   NaN
93900   NaN
Name: children, dtype: float64

In [17]:
hotel.children[hotel.children != hotel.children].index.values

array([93340, 93407, 93419, 93900])

## 3. Removing the unwanted columns

In [18]:
# 열에 있는 값의 70% 이상이 누락되고 누락된 값을 채울 방법이 없는 경우 열을 dataset에서 제거할 수 있습니다
for col in hotel.columns:
    if np.sum(hotel[col].isnull()) > (hotel.shape[0] * 0.7):
        hotel.drop(columns=col, inplace=True, axis=1)

# company 삭제

In [19]:
hotel.drop(columns=["arrival_date_week_number", "arrival_date_year", "arrival_date_month", "arrival_date_day_of_month"],
          inplace=True, axis=1)

In [20]:
hotel.shape

(114465, 30)

## 4. Removing unwanted rows

In [21]:
# 예측하는 값에 따라 필요하지 않은 열을 지울 수 있습니다
# agent는 대리인입니다
hotel.dropna(subset=["agent"], inplace=True)
hotel.shape

(98417, 30)

In [22]:
114465 - 16048

98417

## 5. Filling the missing values of columns

In [23]:
hotel["children"].fillna(value = hotel["children"].mean(), inplace=True)
hotel["children"] = hotel["children"].apply(np.floor)
print(f"Total missing values in children column after filling = {np.sum(hotel.children.isnull())}")

Total missing values in children column after filling = 0


In [26]:
arr = ["market_segment", "distribution_channel", "meal", "country"]
print("No of missing values are")
for x in arr:
    hotel[x].fillna(method="bfill", inplace=True)
    print(f"{x}: {np.sum(hotel[x].isnull())}")

No of missing values are
market_segment: 0
distribution_channel: 0
meal: 0
country: 0


In [27]:
np.sum(hotel.isnull())

hotel                             0
is_canceled                       0
lead_time                         0
stays_in_weekend_nights           0
stays_in_week_nights              0
adults                            0
children                          0
babies                            0
meal                              0
country                           0
market_segment                    0
distribution_channel              0
is_repeated_guest                 0
previous_cancellations            0
previous_bookings_not_canceled    0
reserved_room_type                0
assigned_room_type                0
booking_changes                   0
deposit_type                      0
agent                             0
days_in_waiting_list              0
customer_type                     0
adr                               0
required_car_parking_spaces       0
total_of_special_requests         0
reservation_status                0
reservation_status_date           0
yyyy                        

In [ ]:
df.isnull().sum().sort_values(ascending=False)[:10]